<center><h1>CNN + AE - Escenario experimental 1</header1></center>



## Importación de librerias necesarias

In [1]:
import pandas as pd
import numpy as np

## Cargar el Dataset

In [2]:
datapath = 'https://raw.githubusercontent.com/rcarmas/datasets-Internetworking/main/SelectedFeatures-10s-TOR-NonTOR.csv'
names = ['Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Protocol', 'Flow Duration', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min','Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min','Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min','Active Mean', 'Active Std', 'Active Max', 'Active Min','Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min','label']
#Lee los datos del csv
dataframe = pd.read_csv(datapath,names=names,low_memory=False)

In [3]:
#Visualización de los 5 primeros datos
dataframe.head()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,53913,216.58.208.46,80,6,435,0.0,4597.701149,435.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
1,10.0.2.15,53913,216.58.208.46,80,6,259,0.0,7722.007722,259.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
2,10.0.2.15,53913,216.58.208.46,80,6,891,0.0,2244.668911,891.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
3,10.0.2.15,53913,216.58.208.46,80,6,1074,0.0,1862.197393,1074.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
4,10.0.2.15,53913,216.58.208.46,80,6,315,0.0,6349.206349,315.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR


# Normalización de los datos

In [4]:
# Normalización de los datos
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df

In [5]:
#Muestra el tamaño del dataframe (número de filas y columnas)
print (dataframe.shape)
dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()
print(dataframe.describe())
print (list(dataframe))

(67834, 29)
        Source Port  Destination Port      Protocol  Flow Duration  \
count  67834.000000      67834.000000  67834.000000   6.783400e+04   
mean   37912.753324      11566.395967     12.167291   2.991884e+06   
std    20986.077326      18374.765123      5.459410   4.063005e+06   
min       21.000000         21.000000      6.000000   0.000000e+00   
25%    19305.000000        137.000000      6.000000   4.435975e+04   
50%    43677.000000        443.000000     17.000000   4.108570e+05   
75%    54685.000000      16311.000000     17.000000   7.325550e+06   
max    65534.000000      65514.000000     17.000000   1.000000e+07   

       Flow Bytes/s  Flow Packets/s  Flow IAT Mean  Flow IAT Std  \
count  6.783200e+04    6.783400e+04   6.783400e+04  6.783400e+04   
mean            inf             inf   3.155927e+05  2.209662e+05   
std             NaN             NaN   6.988069e+05  6.409506e+05   
min    0.000000e+00    2.002581e-01   0.000000e+00  0.000000e+00   
25%    1.046553e+

In [6]:
#Obtiene las claves del dataframe (los nombres de las columnas)
keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys) - 1]].copy()
x_normalised = dfNormalize(data_to_process)
print(x_normalised.describe())

           Protocol  Flow Duration  Flow Bytes/s  Flow Packets/s  \
count  67834.000000   67834.000000       67830.0         67828.0   
mean       0.560663       0.299188           0.0             0.0   
std        0.496310       0.406300           0.0             0.0   
min        0.000000       0.000000           0.0             0.0   
25%        0.000000       0.004436           0.0             0.0   
50%        1.000000       0.041086           0.0             0.0   
75%        1.000000       0.732555           0.0             0.0   
max        1.000000       1.000000           0.0             0.0   

       Flow IAT Mean  Flow IAT Std  Flow IAT Max  Flow IAT Min  Fwd IAT Mean  \
count   67834.000000  67834.000000  67834.000000  67834.000000  67834.000000   
mean        0.031600      0.031363      0.089855      0.019491      0.035016   
std         0.069971      0.090973      0.173880      0.057865      0.083937   
min         0.000000      0.000000      0.000000      0.000000     

In [7]:
#Cambiar las etiquetas de los datos de ‘nonTOR’ y ‘TOR’ a 1 y 0 respectivamente.
change_labels = lambda x: 1 if x == 'nonTOR' else 0

In [8]:
#crea una nueva variable llamada y_normalised que contiene los valores de la columna 
#‘label’ del dataframe original después de aplicar la función change_labels.
y_normalised = dataframe['label'].apply(change_labels)

# Implementación del modelo CNN+AE

In [9]:
#Importación de librerías
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D

In [11]:
# Definir la forma de entrada
input_shape = (x_normalised.shape[1], 1)
input_layer = Input(shape=input_shape)

In [12]:
# Parte CNN del modelo
conv1 = Conv1D(32, kernel_size=3, activation='relu', padding='same')(input_layer)
pool1 = MaxPooling1D(pool_size=2)(conv1)
conv2 = Conv1D(16, kernel_size=3, activation='relu', padding='same')(pool1)
encoded = MaxPooling1D(pool_size=2)(conv2)


In [13]:
# Parte AE del modelo
up1 = UpSampling1D(size=2)(encoded)
conv3 = Conv1D(16, kernel_size=3, activation='relu', padding='same')(up1)
up2 = UpSampling1D(size=2)(conv3)
decoded = Conv1D(1, kernel_size=3, activation='linear', padding='same')(up2)

In [14]:
# Crear el modelo
model = Model(inputs=input_layer, outputs=decoded)

In [15]:
#Compilar el modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Entrenamiento del modelo

In [21]:
#Entrenar el modelo
epochs = 10
batch_size = 32
num_samples = len(x_normalised)

for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    for i in range(0, num_samples, batch_size):
        batch = x_normalised[i:i+batch_size]
        batch = np.expand_dims(batch, axis=2)
        loss, mae = model.train_on_batch(batch, batch)
        print(f'  loss: {loss:.6f}, mae: {mae:.6f}')

Se han truncado las últimas 5000 líneas del flujo de salida.
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, mae: nan
  loss: nan, ma

In [24]:
# Evaluar el modelo en el conjunto de validación
x_normalised = np.expand_dims(x_normalised, axis=2)
val_loss, val_mae = model.evaluate(x_normalised, x_normalised, verbose=0)
print(f'Validation loss: {val_loss:.6f}, Validation mae: {val_mae:.6f}')


Validation loss: nan, Validation mae: nan


In [28]:
# Utilizar el modelo para la detección de anomalias en el conjunto de validación
x_normalised = model.predict(x_normalised)
mse = np.mean(np.power(x_normalised - x_normalised, 2), axis=1)
threshold = np.percentile(mse, 95)
y_val_pred = (mse > threshold).astype(int)

2120/2120 [==============================] - 6s 3ms/step


# Resultados

In [31]:
# Calcular la precisión y el recall
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_normalised, y_normalised)
recall = recall_score(y_normalised, y_normalised)
print(f'Precision: {precision:.6f}, Recall: {recall:.6f}')

Precision: 1.000000, Recall: 1.000000
